In [1]:
import pandas as pd
import numpy as np
# 读取数据
df = pd.read_csv('train_dic.csv')
# test = pd.read_csv('test_dic.csv')
# train.shape,test.shape
df.shape

(4007367, 24)

In [2]:
# 定义需要处理的列
columns_to_process = ['warehouse', 'holiday_name', 'L1_category_name_en']
columns_to_replace = ['year', 'name', 'L2_category_name_en', 'L3_category_name_en', 'L4_category_name_en']

# 转换日期格式并提取月份和星期几
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month
df['weekday'] = df['date'].dt.weekday

# 创建“is_weekend”列
df['is_weekend'] = df['weekday'].apply(lambda x: 1 if x >= 5 else 0)

# 循环处理每一列
for col in columns_to_process:
    # 1. 订单总量：计算每列的total_orders平均
    total_orders = df.groupby(col)['total_orders'].mean().reset_index()
    total_orders.rename(columns={'total_orders': f'{col}_total_orders'}, inplace=True)
    df = df.merge(total_orders, on=col, how='left')
    
    # 5. 星期几的销售均值：计算每个星期几的sales平均值
    weekday_mean_values = df.groupby([col, 'weekday'])['sales'].mean().reset_index()
    weekday_mean_values.rename(columns={'sales': f'{col}_weekday_sales_mean'}, inplace=True)
    df = df.merge(weekday_mean_values, on=[col, 'weekday'], how='left')
    
    # 3. 平均值替代编码：计算每列的sales平均值
    mean_values = df.groupby(col)['sales'].mean().reset_index()
    mean_values.rename(columns={'sales': f'{col}_mean'}, inplace=True)
    df = df.merge(mean_values, on=col, how='left')
    # 循环处理每一列
for col in columns_to_replace:
    # 计算每列的sales均值
    mean_values = df.groupby(col)['sales'].mean().reset_index()
    mean_values.rename(columns={'sales': f'{col}_mean'}, inplace=True)
    # 用均值替换原列的值
    df = df.merge(mean_values, on=col, how='left')
df.shape

(4007367, 41)

In [3]:
df.iloc[:, -20:]

,day_cos,is_discount,max_discount,month,weekday,is_weekend,warehouse_total_orders,warehouse_weekday_sales_mean,warehouse_mean,holiday_name_total_orders,holiday_name_weekday_sales_mean,holiday_name_mean,L1_category_name_en_total_orders,L1_category_name_en_weekday_sales_mean,L1_category_name_en_mean,year_mean,name_mean,L2_category_name_en_mean,L3_category_name_en_mean,L4_category_name_en_mean
0,-0.440394,True,0.15312,3,6,1,5517.653329,96.490704,102.934222,6029.969135,102.755561,108.952108,6005.362032,115.557566,121.769964,114.774336,21.746397,90.182361,87.325858,136.896545
1,0.347305,True,0.15025,5,1,0,5517.653329,100.818037,102.934222,6029.969135,103.730718,108.952108,6005.362032,115.002070,121.769964,121.963123,21.746397,90.182361,87.325858,136.896545
2,-0.612106,True,0.15025,12,0,0,5517.653329,102.514483,102.934222,6029.969135,108.962732,108.952108,6005.362032,117.346345,121.769964,121.963123,21.746397,90.182361,87.325858,136.896545
3,0.918958,True,0.20024,4,5,1,5517.653329,98.947674,102.934222,6029.969135,102.683989,108.952108,6005.362032,119.920791,121.769964,99.319686,21.746397,90.182361,87.325858,136.896545
4,0.979530,True,0.15649,4,4,0,5517.653329,112.237905,102.934222,6029.969135,124.816273,108.952108,6005.362032,141.728951,121.769964,103.769209,21.746397,90.182361,87.325858,136.896545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4007362,-0.440394,False,0.00000,6,2,0,9114.406893,140.885796,146.635053,6029.969135,105.607022,108.952108,5917.586699,112.264783,114.425299,99.319686,24.710714,122.492940,151.222107,104.395348
4007363,0.151428,False,0.00000,6,5,1,9114.406893,138.985296,146.635053,6029.969135,102.683989,108.952108,5917.586699,104.534004,114.425299,99.319686,24.710714,122.492940,151.222107,104.395348
4007364,-0.050649,False,0.00000,6,4,0,9114.406893,165.837039,146.635053,6029.969135,124.816273,108.952108,5917.586699,128.909871,114.425299,99.319686,24.710714,122.492940,151.222107,104.395348
4007365,-0.250653,False,0.00000,6,3,0,9114.406893,151.127576,146.635053,6029.969135,113.984084,108.952108,5917.586699,121.480777,114.425299,99.319686,24.710714,122.492940,151.222107,104.395348


In [4]:
missing_values = df.isnull().sum()

# 过滤出有缺失值的列
missing_values = missing_values[missing_values > 0]

# 返回结果
if not missing_values.empty:
    print("存在缺失值的列及缺失个数：")
    print(missing_values)
else:
    print("DataFrame 中没有缺失值。")

DataFrame 中没有缺失值。


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4007367 entries, 0 to 4007366
Data columns (total 41 columns):
 #   Column                                  Dtype         
---  ------                                  -----         
 0   unique_id                               int64         
 1   date                                    datetime64[ns]
 2   warehouse                               object        
 3   total_orders                            float64       
 4   sales                                   float64       
 5   sell_price_main                         float64       
 6   holiday_name                            object        
 7   holiday                                 bool          
 8   shops_closed                            bool          
 9   winter_school_holidays                  bool          
 10  school_holidays                         bool          
 11  product_unique_id                       int64         
 12  name                                    ob

In [6]:
df.columns

Index(['unique_id', 'date', 'warehouse', 'total_orders', 'sales',
       'sell_price_main', 'holiday_name', 'holiday', 'shops_closed',
       'winter_school_holidays', 'school_holidays', 'product_unique_id',
       'name', 'L1_category_name_en', 'L2_category_name_en',
       'L3_category_name_en', 'L4_category_name_en', 'year', 'month_sin',
       'month_cos', 'day_sin', 'day_cos', 'is_discount', 'max_discount',
       'month', 'weekday', 'is_weekend', 'warehouse_total_orders',
       'warehouse_weekday_sales_mean', 'warehouse_mean',
       'holiday_name_total_orders', 'holiday_name_weekday_sales_mean',
       'holiday_name_mean', 'L1_category_name_en_total_orders',
       'L1_category_name_en_weekday_sales_mean', 'L1_category_name_en_mean',
       'year_mean', 'name_mean', 'L2_category_name_en_mean',
       'L3_category_name_en_mean', 'L4_category_name_en_mean'],
      dtype='object')

删除holiday_name name product_unique_id L2_category_name_en L3_category_name_en L4_category_name_en month_sin month_cos

将weekday和month改为字符串

In [7]:
# 删除指定的列
df = df.drop(columns=[
    'holiday_name', 'name', 'product_unique_id', 
    'L2_category_name_en', 'L3_category_name_en', 'L4_category_name_en', 
], errors='ignore')

# 将 weekday 和 month 列转换为字符串类型
df['weekday'] = df['weekday'].astype(str)
df['month'] = df['month'].astype(str)
df['is_weekend'] = df['is_weekend'].astype('bool')
df.shape

(4007367, 35)

In [8]:
df.columns

Index(['unique_id', 'date', 'warehouse', 'total_orders', 'sales',
       'sell_price_main', 'holiday', 'shops_closed', 'winter_school_holidays',
       'school_holidays', 'L1_category_name_en', 'year', 'month_sin',
       'month_cos', 'day_sin', 'day_cos', 'is_discount', 'max_discount',
       'month', 'weekday', 'is_weekend', 'warehouse_total_orders',
       'warehouse_weekday_sales_mean', 'warehouse_mean',
       'holiday_name_total_orders', 'holiday_name_weekday_sales_mean',
       'holiday_name_mean', 'L1_category_name_en_total_orders',
       'L1_category_name_en_weekday_sales_mean', 'L1_category_name_en_mean',
       'year_mean', 'name_mean', 'L2_category_name_en_mean',
       'L3_category_name_en_mean', 'L4_category_name_en_mean'],
      dtype='object')

In [9]:
# 选出 object 类型的列
object_columns = df.select_dtypes(include=['object']).columns
# 对 object 列进行独热编码，并设置 drop_first=True
df = pd.get_dummies(df, columns=object_columns, drop_first=True)

In [10]:
df.columns

Index(['unique_id', 'date', 'total_orders', 'sales', 'sell_price_main',
       'holiday', 'shops_closed', 'winter_school_holidays', 'school_holidays',
       'month_sin', 'month_cos', 'day_sin', 'day_cos', 'is_discount',
       'max_discount', 'is_weekend', 'warehouse_total_orders',
       'warehouse_weekday_sales_mean', 'warehouse_mean',
       'holiday_name_total_orders', 'holiday_name_weekday_sales_mean',
       'holiday_name_mean', 'L1_category_name_en_total_orders',
       'L1_category_name_en_weekday_sales_mean', 'L1_category_name_en_mean',
       'year_mean', 'name_mean', 'L2_category_name_en_mean',
       'L3_category_name_en_mean', 'L4_category_name_en_mean',
       'warehouse_Budapest_1', 'warehouse_Frankfurt_1', 'warehouse_Munich_1',
       'warehouse_Prague_1', 'warehouse_Prague_2', 'warehouse_Prague_3',
       'L1_category_name_en_Fruit and vegetable',
       'L1_category_name_en_Meat and fish', 'year_year_2021', 'year_year_2022',
       'year_year_2023', 'year_year_2024',

In [11]:
df.to_csv('train_dic_processed.csv', index=False)

In [12]:
# for df in [train, test]:    
# # 检查缺失值
#     missing_values = df.isnull().sum()

#     # 过滤出有缺失值的列
#     missing_values = missing_values[missing_values > 0]

#     # 返回结果
#     if not missing_values.empty:
#         print("存在缺失值的列及缺失个数：")
#         print(missing_values)
#     else:
#         print("DataFrame 中没有缺失值。")